In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os

import numpy as np
import pandas as pd
import atecml.data
from tqdm import tqdm

In [2]:
train_df = atecml.data.load_train()
val_df = atecml.data.load_test()
train_df.loc[train_df.label == 0, 'Fraud'] = 0
train_df.loc[train_df.label != 0, 'Fraud'] = 1

In [3]:
predictors = [x for x in train_df.columns if x not in atecml.data.NOT_FEATURE_SUM]


target = 'Fraud'
DateFold={}

DateFold[0] = set(atecml.data.filter_date(train_df,start_date='2017-09-05',end_date='2017-09-12').index)
DateFold[1] = set(atecml.data.filter_date(train_df,start_date='2017-09-13',end_date='2017-09-20').index)
DateFold[2] = set(atecml.data.filter_date(train_df,start_date='2017-09-21',end_date='2017-09-28').index)
DateFold[3] = set(atecml.data.filter_date(train_df,start_date='2017-09-29',end_date='2017-10-06').index)
DateFold[4] = set(atecml.data.filter_date(train_df,start_date='2017-10-07',end_date='2017-10-14').index)
DateFold[5] = list(atecml.data.filter_date(train_df,start_date='2017-10-15',end_date='2017-11-24').index)

all_list = set(train_df.index) - set(DateFold[5])
len(all_list),len(DateFold[5])

(634284, 360447)

In [5]:
X = np.array(train_df[predictors])
Y = np.array(train_df[target])

val_index = DateFold[5] 
train_index = list(all_list)

X_train = X[train_index]
y_train = Y[train_index]

X_val = X[val_index]
y_val = Y[val_index]
    
X_test = np.array(val_df[predictors])
print(X_train.shape, y_train.shape, X_test.shape)

(634284, 163) (634284,) (500538, 163)


In [ ]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,free_raw_data=False)

print('设置参数')
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'binary_logloss',
          }

### 交叉验证(调参)
print('交叉验证')
min_merror = float('Inf')
best_params = {}

# 准确率
print("调参1：提高准确率")
for num_leaves in range(20,300,5):
    for max_depth in range(3,10,1):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth

        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=2018,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
            
        mean_merror = pd.Series(cv_results['auc-mean']).min()
        boost_rounds = pd.Series(cv_results['auc-mean']).argmin()
            
        if mean_merror < min_merror:
            min_merror = mean_merror
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth
            
params['num_leaves'] = best_params['num_leaves']
params['max_depth'] = best_params['max_depth']

设置参数
交叉验证
调参1：提高准确率
[1]	cv_agg's auc: 0.780309 + 0.0223346
[2]	cv_agg's auc: 0.920521 + 0.00380023
[3]	cv_agg's auc: 0.931247 + 0.00287506
[4]	cv_agg's auc: 0.93438 + 0.00264701
[5]	cv_agg's auc: 0.939759 + 0.00256712
[6]	cv_agg's auc: 0.942115 + 0.00283327
[7]	cv_agg's auc: 0.943406 + 0.00294346
[8]	cv_agg's auc: 0.944551 + 0.00308664
[9]	cv_agg's auc: 0.948084 + 0.00498538
[10]	cv_agg's auc: 0.951447 + 0.00325723
[11]	cv_agg's auc: 0.95219 + 0.00346694
[12]	cv_agg's auc: 0.953529 + 0.00350649
[13]	cv_agg's auc: 0.954702 + 0.00276043
[14]	cv_agg's auc: 0.956026 + 0.00242048
[15]	cv_agg's auc: 0.956658 + 0.00222616
[16]	cv_agg's auc: 0.957878 + 0.00186235
[17]	cv_agg's auc: 0.959245 + 0.00209138
[18]	cv_agg's auc: 0.959981 + 0.00207071
[19]	cv_agg's auc: 0.960439 + 0.00212486
[20]	cv_agg's auc: 0.961139 + 0.00200165
[21]	cv_agg's auc: 0.961614 + 0.00199522
[22]	cv_agg's auc: 0.962113 + 0.0019604
[23]	cv_agg's auc: 0.962551 + 0.00183618
[24]	cv_agg's auc: 0.963084 + 0.00174219
[25]	cv_a

[3]	cv_agg's auc: 0.94792 + 0.00161537
[4]	cv_agg's auc: 0.952531 + 0.00248375
[5]	cv_agg's auc: 0.956931 + 0.00315291
[6]	cv_agg's auc: 0.959513 + 0.00289498
[7]	cv_agg's auc: 0.961579 + 0.0025666
[8]	cv_agg's auc: 0.96327 + 0.00250139
[9]	cv_agg's auc: 0.964191 + 0.00265624
[10]	cv_agg's auc: 0.964976 + 0.00253923
[11]	cv_agg's auc: 0.965681 + 0.00264462
[12]	cv_agg's auc: 0.966578 + 0.00255333
[13]	cv_agg's auc: 0.967309 + 0.00258391
[14]	cv_agg's auc: 0.967829 + 0.00271227
[15]	cv_agg's auc: 0.968172 + 0.00268331
[16]	cv_agg's auc: 0.968791 + 0.00250315
[17]	cv_agg's auc: 0.969188 + 0.00257745
[18]	cv_agg's auc: 0.969546 + 0.00247529
[19]	cv_agg's auc: 0.969903 + 0.00255093
[20]	cv_agg's auc: 0.970394 + 0.00227626
[21]	cv_agg's auc: 0.970953 + 0.00243009
[22]	cv_agg's auc: 0.971558 + 0.00221884
[23]	cv_agg's auc: 0.971931 + 0.00207546
[24]	cv_agg's auc: 0.97226 + 0.00207398
[25]	cv_agg's auc: 0.972594 + 0.00196659
[26]	cv_agg's auc: 0.972796 + 0.00192781
[27]	cv_agg's auc: 0.973099

[5]	cv_agg's auc: 0.953253 + 0.00263796
[6]	cv_agg's auc: 0.956454 + 0.00136544
[7]	cv_agg's auc: 0.959019 + 0.00221268
[8]	cv_agg's auc: 0.962718 + 0.00283988
[9]	cv_agg's auc: 0.964118 + 0.00260265
[10]	cv_agg's auc: 0.964949 + 0.00265733
[11]	cv_agg's auc: 0.965697 + 0.00259528
[12]	cv_agg's auc: 0.966642 + 0.00248728
[13]	cv_agg's auc: 0.967403 + 0.0024459
[14]	cv_agg's auc: 0.967931 + 0.00245167
[15]	cv_agg's auc: 0.968383 + 0.00268311
[16]	cv_agg's auc: 0.969122 + 0.00250937
[17]	cv_agg's auc: 0.969483 + 0.00243884
[18]	cv_agg's auc: 0.969936 + 0.00224047
[19]	cv_agg's auc: 0.970337 + 0.00229477
[20]	cv_agg's auc: 0.970786 + 0.00242763
[21]	cv_agg's auc: 0.971491 + 0.00257971
[22]	cv_agg's auc: 0.972391 + 0.00191965
[23]	cv_agg's auc: 0.972699 + 0.0020054
[24]	cv_agg's auc: 0.973096 + 0.00202797
[25]	cv_agg's auc: 0.973344 + 0.00206426
[26]	cv_agg's auc: 0.973774 + 0.00193938
[27]	cv_agg's auc: 0.974057 + 0.00208999
[28]	cv_agg's auc: 0.974268 + 0.00205685
[29]	cv_agg's auc: 0.97

[7]	cv_agg's auc: 0.956405 + 0.00418262
[8]	cv_agg's auc: 0.960549 + 0.00541226
[9]	cv_agg's auc: 0.96221 + 0.00389026
[10]	cv_agg's auc: 0.962842 + 0.00389009
[11]	cv_agg's auc: 0.964083 + 0.00331952
[12]	cv_agg's auc: 0.964746 + 0.00342009
[13]	cv_agg's auc: 0.966071 + 0.00358503
[14]	cv_agg's auc: 0.96736 + 0.00262542
[15]	cv_agg's auc: 0.967929 + 0.0029352
[16]	cv_agg's auc: 0.968409 + 0.00293735
[17]	cv_agg's auc: 0.969051 + 0.00281569
[18]	cv_agg's auc: 0.969603 + 0.00266531
[19]	cv_agg's auc: 0.97028 + 0.00245843
[20]	cv_agg's auc: 0.970629 + 0.00250413
[21]	cv_agg's auc: 0.971095 + 0.00251844
[22]	cv_agg's auc: 0.971572 + 0.0023357
[23]	cv_agg's auc: 0.972221 + 0.00223217
[24]	cv_agg's auc: 0.972555 + 0.00227235
[25]	cv_agg's auc: 0.973099 + 0.00210285
[26]	cv_agg's auc: 0.973374 + 0.0021066
[27]	cv_agg's auc: 0.973745 + 0.00205737
[28]	cv_agg's auc: 0.974044 + 0.00193041
[29]	cv_agg's auc: 0.974364 + 0.00192681
[30]	cv_agg's auc: 0.974618 + 0.00192295
[31]	cv_agg's auc: 0.9749

[10]	cv_agg's auc: 0.960231 + 0.00275609
[11]	cv_agg's auc: 0.961736 + 0.00297101
[12]	cv_agg's auc: 0.962876 + 0.00272729
[13]	cv_agg's auc: 0.963644 + 0.00275236
[14]	cv_agg's auc: 0.964251 + 0.00278124
[15]	cv_agg's auc: 0.964898 + 0.00286975
[16]	cv_agg's auc: 0.965579 + 0.00251336
[17]	cv_agg's auc: 0.965956 + 0.00256921
[18]	cv_agg's auc: 0.966272 + 0.00262971
[19]	cv_agg's auc: 0.967397 + 0.00257617
[20]	cv_agg's auc: 0.968142 + 0.00224649
[21]	cv_agg's auc: 0.968577 + 0.00216236
[22]	cv_agg's auc: 0.969086 + 0.00192399
[23]	cv_agg's auc: 0.969495 + 0.00180173
[24]	cv_agg's auc: 0.969777 + 0.00183801
[25]	cv_agg's auc: 0.970052 + 0.0017201
[26]	cv_agg's auc: 0.970361 + 0.00180195
[27]	cv_agg's auc: 0.970561 + 0.00181383
[28]	cv_agg's auc: 0.970784 + 0.00170425
[29]	cv_agg's auc: 0.970986 + 0.00170857
[30]	cv_agg's auc: 0.971161 + 0.00171706
[31]	cv_agg's auc: 0.971414 + 0.00167651
[32]	cv_agg's auc: 0.971581 + 0.00167643
[33]	cv_agg's auc: 0.971745 + 0.00172171
[34]	cv_agg's auc

[13]	cv_agg's auc: 0.96908 + 0.00294032
[14]	cv_agg's auc: 0.969455 + 0.00285724
[15]	cv_agg's auc: 0.969887 + 0.00291856
[16]	cv_agg's auc: 0.970287 + 0.00293232
[17]	cv_agg's auc: 0.970633 + 0.00284304
[18]	cv_agg's auc: 0.971029 + 0.00281772
[19]	cv_agg's auc: 0.971267 + 0.00282904
[20]	cv_agg's auc: 0.971552 + 0.00275304
[21]	cv_agg's auc: 0.972146 + 0.00250838
[22]	cv_agg's auc: 0.973026 + 0.00257322
[23]	cv_agg's auc: 0.973809 + 0.00207147
[24]	cv_agg's auc: 0.974089 + 0.00202638
[25]	cv_agg's auc: 0.974344 + 0.00202109
[26]	cv_agg's auc: 0.974536 + 0.00205862
[27]	cv_agg's auc: 0.974714 + 0.0021079
[28]	cv_agg's auc: 0.974965 + 0.00206353
[29]	cv_agg's auc: 0.975196 + 0.00205846
[30]	cv_agg's auc: 0.975344 + 0.00203102
[31]	cv_agg's auc: 0.97557 + 0.00199476
[32]	cv_agg's auc: 0.975758 + 0.00196597
[33]	cv_agg's auc: 0.975865 + 0.00193142
[34]	cv_agg's auc: 0.976015 + 0.00198755
[35]	cv_agg's auc: 0.976146 + 0.00194328
[36]	cv_agg's auc: 0.976336 + 0.00192971
[37]	cv_agg's auc: 

[15]	cv_agg's auc: 0.970154 + 0.00243745
[16]	cv_agg's auc: 0.970729 + 0.0024784
[17]	cv_agg's auc: 0.971302 + 0.00245704
[18]	cv_agg's auc: 0.971549 + 0.00249585
[19]	cv_agg's auc: 0.971854 + 0.0025261
[20]	cv_agg's auc: 0.972093 + 0.00244132
[21]	cv_agg's auc: 0.972827 + 0.00209557
[22]	cv_agg's auc: 0.973254 + 0.00210607
[23]	cv_agg's auc: 0.973678 + 0.0019577
[24]	cv_agg's auc: 0.974138 + 0.00185774
[25]	cv_agg's auc: 0.9745 + 0.00172635
[26]	cv_agg's auc: 0.974933 + 0.00172937
[27]	cv_agg's auc: 0.975184 + 0.00158922
[28]	cv_agg's auc: 0.97548 + 0.00156182
[29]	cv_agg's auc: 0.975649 + 0.00167252
[30]	cv_agg's auc: 0.975895 + 0.0017101
[31]	cv_agg's auc: 0.976051 + 0.00173634
[32]	cv_agg's auc: 0.976172 + 0.00168014
[33]	cv_agg's auc: 0.976352 + 0.00168663
[34]	cv_agg's auc: 0.976481 + 0.00172061
[35]	cv_agg's auc: 0.976587 + 0.00172199
[36]	cv_agg's auc: 0.97669 + 0.00174085
[37]	cv_agg's auc: 0.976801 + 0.0017125
[38]	cv_agg's auc: 0.976932 + 0.00169183
[39]	cv_agg's auc: 0.9770